In [1]:
import json
import os

In [8]:
# загрузка файла json с сегментацией загрязнений/утрат

with open('dirt_output.json', 'r') as f:
# with open('lacunae_output.json', 'r') as f:
  data = json.load(f)


In [9]:
# распаковка необходимой информацию из json файла - изображения и аннотации к ним
images = data['images']
annotations = data['annotations']
categories = {c['id']: c['name'] for c in data['categories']} #соотносим id категории (класса) с именем категории (класса)

In [12]:
print(images[1])

{'license': 0, 'file_name': 'ad000122.png', 'coco_url': 'n/a', 'height': 599, 'width': 710, 'date_captured': '', 'flickr_url': 'n/a', 'darwin_url': 'https://darwin.v7labs.com/api/v2/teams/rsl/uploads/c3ba22ab-75c4-4cff-a2f5-342f8d263d68', 'darwin_workview_url': 'https://darwin.v7labs.com/workview?dataset=653839&item=0189212b-7763-d0aa-0f50-a3de1113c249', 'id': 2877091105, 'tag_ids': []}


In [13]:
print(categories)

{2556944822: 'dirt'}


In [14]:
#тестовая проверка правильности отображения координат
test_list = [414.0, 229.0, 417.0, 230.0, 418.0, 230.0, 419.0, 232.0]
for i in test_list:
    if isinstance(i, str or dict):
        continue
    elif not test_list.index(i) %2:
        print(str(i) + ' x')
    else: 
        print(str(i) + ' y')


414.0 x
229.0 y
417.0 x
230.0 y
418.0 x
230.0 y
419.0 x
232.0 y


In [23]:
# функция добавления информации в файл
def write_to_file(file, str):
    with open(file, 'a', encoding = 'utf-8') as f:
        f.write(str)

In [24]:
# проверка значений внутри файла JSON и запись в файл строк с некорректными данными
#---- MAIN -----

for image in images: #для каждой записи в данных изображения
    segment_boxes = [] #пустой список для конвертированных данных
    file_name = image["file_name"].replace('.png', '') #данные имени файла без расширения
    for annotation in annotations: #для каждой записи в данных сегментации
        if annotation['image_id'] == image['id']: #если id изображения соответствует id аннотации
            #print(annotation["segmentation"])
            #print(image["height"])
            #print(image["width"])
            bbox = annotation["bbox"] #создаем переменную, содержащую данные рамки bbox - координаты x y w h
            # нормализация координат
            x, y, w, h = bbox
            file_name = 'file_name  ' + str(image["file_name"])
            id_annot = 'id annotation  ' + str(annotation['image_id'])
            id_img = 'id image  ' + str(image['id'])
            img_w = 'img_width(x) ' + str(image['width'])
            img_h = 'img_height(y) ' + str(image['height'])
            x = x / image['width']
            y = y / image['height']
            w = w / image['width']
            h = h / image['height']
            k_x = 1/image['width']
            k_y = 1/image['height']
            # print('k_y'+str(k_y))
            for i in annotation["segmentation"]: #для элемента списка сегментов (список в списке)
                
                if isinstance(i, dict): # исключение, если в записи есть полигон семантического сегмента - маска (словарь с ключом count), а не сущностного 
                    continue
                else:
                    # print(i)
                    max_coord = 'макc_значения в списке '+ str(max(i)) #выводим максимальное значение координаты
                    # for n in i:
                    ind_max = 'индекс макс_значения - чет=x, нечет=y ' + str(i.index(max(i))) #выводим индекс максимального значения, где четные значения относятся к x, а нечетные к y
                    final_str = f'{file_name}\n{id_annot}\n{id_img}\n{img_w}\n{img_h}\n{max_coord}\n{ind_max}\n'
                    write_to_file('info.txt', final_str) #запись в файл
                    maximum = max(i)
                    ind = i.index(max(i))
                    if isinstance(i, str or dict): # исключение, если в записи есть полигон семантического сегмента - маска (словарь с ключом count), а не сущностного
                        continue
                    elif not ind % 2: #если индекс не кратен 2, получаем нормальзованный x (значение от 0 до 1)
                        x_norm = maximum / image['width']
                        x_norm_k = maximum * k_x
                        x_norm_str = 'нормализованный x | коэффициент ' + str(x_norm) + ' '+ str(x_norm_k) #выводим запись нормализованных значений x
                        write_to_file('info.txt', f'{x_norm_str}\n') #запись в файл
                        if x_norm_k > 1.0: # если нормализованный x > 1.0, выводим запись об ошибке
                            error_x = 'ERROR'
                            write_to_file('info.txt', f'{error_x}\n') #запись в файл
                    else:   #если индекс кратен 2
                        y_norm = maximum / image['height'] #получаем нормализованный y (значение от 0 до 1)
                        y_norm_k = maximum * k_y
                        y_norm_str = 'нормализованный y | коэффициент ' + str(y_norm) + ' '+ str(y_norm_k) #выводим нормализованное значение по y
                        write_to_file('info.txt', f'{y_norm_str}\n')
                        if y_norm_k > 1.0: # если нормализованный x > 1.0, 
                            # выводим запись об ошибке
                            error_y = 'ERROR'
                            write_to_file('info.txt', f'{error_y}\n')

In [34]:
path_darwin_to_yolo = 'darwin_to_yolo'
# os.mkdir('darwin_to_yolo')
# os.mkdir('pylabel_to_yolo')
os.getcwd()
# os.chdir(r'home_path')
# os.listdir()

In [42]:
# ----- MAIN ------
# ИСПРАВЛЕНИЕ КООРДИНАТ ФАЙЛА JSON с учетом размера файла изображения 
# ЗАПИСЬ ФАЙЛОВ С КООРДИНАТАМИ В ФОРМАТЕ INT (В КАЧЕСТВЕ КООРДИНАТ ДЛЯ НОРМАЛИЗАЦИИ И ИСПРАВЛЕНИЯ ИСПОЛЬЗУЮТСЯ ЦЕЛОЧИСЛЕННЫЕ ЗНАЧЕНИЯ)
# Более корректный вариант координат

for image in images: #для каждой записи в данных изображения
    segment_boxes = [] #пустой список для конвертированных данных
    file_name = image["file_name"].replace('.png', '') #данные имени файла без расширения
    for annotation in annotations: #для каждой записи в аннотациях
        if annotation['image_id'] == image['id']: #если id изображения соответствует id аннотации
            bbox = annotation["bbox"] #создаем переменную, содержащую данные рамки bbox - координаты x y w h
            # нормализация координат
            x, y, w, h = bbox
            # print('file_name  ' + str(image["file_name"]))
            
            x = x / float(image['width'])
            y = y / float(image['height'])
            w = w / float(image['width'])
            h = h / float(image['height'])
            k_x = 1/float(image['width'])
            k_y = 1/float(image['height'])
            # print('k_y'+str(k_y))
            
            for i in annotation["segmentation"]: #для элемента сегмента (список в списке)
                if isinstance(i, dict): # исключение, если в записи есть полигон семантического сегмента - маска (словарь с ключом count), а не сущностного 
                    continue
                else:
                    # print(i)
                    temp_list = []
                    # print("input_str" + str(i))
                    for ind, meaning in enumerate(i):
                        # print(elem[1])
                        if isinstance(meaning, str or dict): # исключение, если в записи есть полигон семантического сегмента - маска (словарь с ключом count), а не сущностного 
                            continue
                        elif not ind % 2 and meaning > float(image["width"]):
                            meaning = float(image["width"]) - 1
                            # print(str(meaning) + "oops_x")
                            temp_list.append(int(meaning))
                        elif ind % 2 and meaning > float(image["height"]):
                            meaning = float(image["height"]) - 1
                            # print(str(meaning) + "oops_y")
                            temp_list.append(int(meaning))
                        else:
                            temp_list.append(int(meaning))
                    # print(temp_list)
                    
                    # нормализация скорректированных координат из temp_list
                    norm_list = []
                    for ind, a in enumerate(temp_list):
                        if not ind % 2:
                            x_norm = a*k_x
                            norm_list.append(f"{x_norm:.3}")
                        else:
                            y_norm = a*k_y
                            norm_list.append(f"{y_norm:.3}")
                    
                    category_id = 0 #dirt
                    # category_id = 1 #lacunae
                    
                    # segment_box_str = f"{category_id} {(x + w/2):.4} {(y + h/2):.4} {w:.4} {h:.4} {norm_list}" #вместе с bbox
                    segment_box_str = f"{category_id} {norm_list}" #только id категории и координаты сегментации без bbox
                    # print(segment_box_str)
                    segment_boxes.append(segment_box_str)
                    # print(segment_boxes)
                    
                    # запись в файл результатов преобразования
                    with open(os.path.join(f"{path_darwin_to_yolo}", f"{file_name}.txt"), 'w') as f:
                    # with open(os.path.join(f"{path_pylabel_to_yolo}", f"{file_name}.txt"), 'w') as f:
                        for text in segment_boxes:
                            clear_str = text.replace("[", "").replace(",", "").replace("]", "").replace("'", "")
                    #         print(clear_str)
                            f.write(f"{clear_str}\n")

In [8]:
#попытка конвертировать darwin с помощью существующего пакета в Python py-darwin 
# pip install darwin-py

# from darwin.cli_functions import convert

# annotation_format = "yolo"
# files = f"{os.getcwd()}/lacunae_output.json"
# output_dir = f"{os.getcwd()}/to_yolo"

# convert(annotation_format, files, output_dir)

Converting annotations...


UnrecognizableFileEncoding: Unable to load file \$Recycle.Bin\S-1-5-21-3374549764-2208607457-1609041802-1001\$I0JMZ5J.json with any encodings: ['utf-8', 'utf-16', 'utf-32', 'ascii']